###Ingest constructors data from bronze to silver

0. Import configuration notebook and widgets

In [0]:
%run ../Includes/Configuration

In [0]:
dbutils.widgets.text('p_data_source', '', 'Data Source')
v_data_source = dbutils.widgets.get('p_data_source')

1. Imports

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, current_timestamp, lit

2. Create schema

In [0]:
constructors_schema = StructType(
    fields=[
        StructField("constructorId", IntegerType(), False),
        StructField("constructorRef", StringType(), False),
        StructField("name", StringType(), False),
        StructField("nationality", StringType(), True),
        StructField("url", StringType(), True)
    ]
)

3. Read constructors.json file

In [0]:
constructors_raw_df = spark.read \
    .format("json") \
    .schema(constructors_schema) \
    .load(f"{bronze_container_path}/constructors.json")

4. Transform the circuit dataframe

In [0]:
constructors_df = constructors_raw_df \
        .select(
                col("constructorId").alias("constructor_id"),
                col("constructorRef").alias("constructor_ref"),
                col("name"),
                col("nationality"),
                current_timestamp().alias("ingestion_timestamp")
        ) \
        .withColumn("data_source", lit(v_data_source))

5. Write the dataframe to silver layer

In [0]:
constructors_df.write.mode('overwrite').parquet(f"{silver_container_path}/constructors")